In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import pandas as pd

# Параметры системы
m = 1.0        # масса (кг)
k = 10.0       # жесткость пружины (Н/м)
alpha = 0.5    # коэффициент линейного сопротивления (Н·с/м)
beta = 0.1     # коэффициент квадратичного сопротивления (Н·с²/м²)

# Начальные условия
x0 = 1.0       # начальное смещение (м)
v0 = 0.0       # начальная скорость (м/с)

# Производные параметры
omega0 = np.sqrt(k/m)           # собственная частота
gamma = alpha/(2*m)              # коэффициент затухания
omega_d = np.sqrt(omega0**2 - gamma**2) if gamma < omega0 else 0  # частота затухающих колебаний

# Временной интервал
t_max = 20.0
t_span = (0, t_max)
t_eval = np.linspace(0, t_max, 1000)  # ВАЖНО: определяем t_eval здесь!


In [ ]:
# Без сопротивления
def system_no_damping(t, state):
    x, v = state
    dxdt = v
    dvdt = -(k/m) * x
    return [dxdt, dvdt]

solution_no_damping = solve_ivp(
    system_no_damping,
    t_span,
    [x0, v0],
    method='RK45',
    dense_output=True,
    rtol=1e-10
)

x_num_no_damping = solution_no_damping.sol(t_eval)[0]
v_num_no_damping = solution_no_damping.sol(t_eval)[1]


$$\begin{cases}
\frac{dx}{dt} = v \\
\frac{dv}{dt} = -\frac{k}{m}x
\end{cases}$$


In [ ]:
# С линейным сопротивлением
def system_linear_damping(t, state):
    x, v = state
    dxdt = v
    dvdt = -(k/m) * x - (alpha/m) * v
    return [dxdt, dvdt]

solution_linear = solve_ivp(
    system_linear_damping,
    t_span,
    [x0, v0],
    method='RK45',
    dense_output=True,
    rtol=1e-10
)

x_num_linear = solution_linear.sol(t_eval)[0]
v_num_linear = solution_linear.sol(t_eval)[1]


$$\begin{cases}
\frac{dx}{dt} = v \\
\frac{dv}{dt} = -\frac{k}{m}x - \frac{\alpha}{m}v
\end{cases}$$


In [ ]:
# Аналитические решения
# Без затухания
A = x0  # амплитуда (при v0=0)
phi = 0  # начальная фаза (при v0=0)

x_analytical_no_damping = A * np.cos(omega0 * t_eval + phi)
v_analytical_no_damping = -A * omega0 * np.sin(omega0 * t_eval + phi)

# С линейным затуханием (для слабого затухания)
if gamma < omega0:
    x_analytical_linear = np.exp(-gamma * t_eval) * (
        x0 * np.cos(omega_d * t_eval) + 
        ((v0 + gamma * x0) / omega_d) * np.sin(omega_d * t_eval)
    )
    v_analytical_linear = np.exp(-gamma * t_eval) * (
        -(x0 * omega_d + (v0 + gamma * x0) * gamma / omega_d) * np.sin(omega_d * t_eval) +
        (v0 + gamma * x0 - gamma * x0) * np.cos(omega_d * t_eval)
    ) - gamma * x_analytical_linear
elif gamma > omega0:
    # Сильное затухание
    lambda1 = -gamma + np.sqrt(gamma**2 - omega0**2)
    lambda2 = -gamma - np.sqrt(gamma**2 - omega0**2)
    A = (v0 - lambda2 * x0) / (lambda1 - lambda2)
    B = (lambda1 * x0 - v0) / (lambda1 - lambda2)
    x_analytical_linear = A * np.exp(lambda1 * t_eval) + B * np.exp(lambda2 * t_eval)
    v_analytical_linear = A * lambda1 * np.exp(lambda1 * t_eval) + B * lambda2 * np.exp(lambda2 * t_eval)
else:
    # Критическое затухание
    x_analytical_linear = np.exp(-gamma * t_eval) * (x0 + (v0 + gamma * x0) * t_eval)
    v_analytical_linear = np.exp(-gamma * t_eval) * (v0 - gamma * (v0 + gamma * x0) * t_eval)


$$\begin{cases}
x(t) = A\cos(\omega_0 t + \phi) \\
v(t) = -A\omega_0\sin(\omega_0 t + \phi)
\end{cases}$$

$$\begin{cases}
x(t) = e^{-\gamma t}(A\cos(\omega_d t) + B\sin(\omega_d t)) \\
v(t) = e^{-\gamma t}(-A\omega_d\sin(\omega_d t) + B\omega_d\cos(\omega_d t)) - \gamma x(t)
\end{cases}$$


In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(t_eval, x_num_no_damping, 'b-', linewidth=2, label='Без демпфирования (численное)')
plt.plot(t_eval, x_analytical_no_damping, 'b--', linewidth=2, label='Без демпфирования (аналитическое)')
plt.plot(t_eval, x_num_linear, 'r-', linewidth=2, label='С демпфированием (численное)')
plt.plot(t_eval, x_analytical_linear, 'r--', linewidth=2, label='С демпфированием (аналитическое)')
plt.xlabel('Время t, с')
plt.ylabel('Координата x, м')
plt.title('Сравнение численных и аналитических решений')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

results = pd.DataFrame({
    '': ['Без демпфирования (численное)', 'Без демпфирования (аналитическое)', 
         'С демпфированием (численное)', 'С демпфированием (аналитическое)'],
    'Макс. смещение (м)': [
        f"{np.max(np.abs(x_num_no_damping)):.1f}",
        f"{np.max(np.abs(x_analytical_no_damping)):.1f}",
        f"{np.max(np.abs(x_num_linear)):.1f}",
        f"{np.max(np.abs(x_analytical_linear)):.1f}"
    ],
    'Период (с)': [
        f"{2*np.pi/omega0:.6f}",
        f"{2*np.pi/omega0:.6f}",
        f"{2*np.pi/omega_d:.6f}" if gamma < omega0 else "N/A",
        f"{2*np.pi/omega_d:.6f}" if gamma < omega0 else "N/A"
    ],
    'Амплитуда (м)': [
        f"{np.max(np.abs(x_num_no_damping)):.1f}",
        f"{np.max(np.abs(x_analytical_no_damping)):.1f}",
        f"{np.max(np.abs(x_num_linear)):.1f}",
        f"{np.max(np.abs(x_analytical_linear)):.1f}"
    ]
})

results


 и н

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import pandas as pd

# ==========================================
# ПАРАМЕТРЫ СИСТЕМЫ
# ==========================================
m = 1.0        # масса (кг)
k = 10.0       # жесткость пружины (Н/м)
alpha = 0.5    # коэффициент линейного сопротивления (Н·с/м)
beta = 0.1     # коэффициент квадратичного сопротивления (Н·с²/м²)

# Начальные условия
x0 = 1.0       # начальное смещение (м)
v0 = 0.0       # начальная скорость (м/с)

# Временной интервал
t_max = 20.0
t_span = (0, t_max)
t_eval = np.linspace(0, t_max, 1000)  # ВАЖНО: определяем t_eval здесь!

# Производные параметры
omega0 = np.sqrt(k/m)           # собственная частота
gamma = alpha/(2*m)              # коэффициент затухания
omega_d = np.sqrt(omega0**2 - gamma**2) if gamma < omega0 else 0  # частота затухающих колебаний

print(f"Параметры системы:")
print(f"ω₀ = {omega0:.3f} рад/с (собственная частота)")
print(f"T₀ = {2*np.pi/omega0:.3f} с (период собственных колебаний)")
print(f"γ = {gamma:.3f} 1/с (коэффициент затухания)")
if gamma < omega0:
    print(f"ω_d = {omega_d:.3f} рад/с (частота затухающих колебаний)")
    print(f"T_d = {2*np.pi/omega_d:.3f} с (период затухающих колебаний)")
    print("Режим: слабое затухание (колебательный)")
elif gamma > omega0:
    print("Режим: сильное затухание (апериодический)")
else:
    print("Режим: критическое затухание")
